In [79]:
%cd ..

/Users/nguyenhuyhai/20221/lab_blockchain


In [4]:
from pyspark.sql import SparkSession, DataFrame, Window
import pyspark.sql.functions as F

In [5]:
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()

23/01/30 09:52:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
def preprocess(df):
    num_val_blocks_df = (
            df.groupBy("block_height")
            .agg({"operator_address": "count"})
            .withColumnRenamed("count(operator_address)", "validators_count_per_block")
        )

In [69]:
df = spark.createDataFrame(
    [
        ("a",1,2,0.1,1),
        ("a",2,5,0.1,2),
        ("a",3,1,0.1,3),
        ("a",4,4,0.1,4),
        ("a",5,5,0.1,5),
        ("a",6,6,0.1,6),
        ("a",7,8,0.1,1),
        ("a",8,3,0.1,1),
        ("a",9,1,0.1,2),
        ("a",10,2,0.1,3),
        ("b",1,4,0.2,2),
        ("b",2,4,0.2,1), 
        ("b",3,20,0.2,2), 
        ("b",4,12,0.2,4), 
        ("b",5,12,0.2,3), 
        ("b",6,4,0.2,0), 
        ("b",7,4,0.2,2), 
        ("b",8,10,0.2,3), 
        ("b",9,4,0.2,4), 
        ("b",10,4,0.2,5),
        ("c",1,4,0.3,2),
        ("c",2,4,0.3,1), 
        ("c",3,4,0.3,2), 
        ("c",4,4,0.3,3),
        ("c",5,7,0.3,7),
        ("c",6,4,0.3,3), 
        ("c",7,4,0.3,3), 
        ("c",8,4,0.3,4), 
        ("c",9,4,0.3,2), 
        ("c",10,4,0.3,1),
        ("d",1,4,0.4,4),
        ("d",2,4,0.4,2), 
        ("d",3,4,0.4,1), 
        ("d",4,4,0.4,3),
        ("d",5,7,0.4,7),
        ("d",6,4,0.4,4), 
        ("d",7,4,0.4,3), 
        ("d",8,4,0.4,2), 
        ("d",9,4,0.4,1), 
        ("d",10,4,0.4,0),
    
    ],

    ["operator_address", "block_height", "delegators_token", "commission_rate", "predict"]
    
    
    
    
    
    
    )

In [70]:
df1 = df.orderBy("operator_address", "block_height")

In [71]:
df1.show(200)

+----------------+------------+----------------+---------------+-------+
|operator_address|block_height|delegators_token|commission_rate|predict|
+----------------+------------+----------------+---------------+-------+
|               a|           1|               2|            0.1|      1|
|               a|           2|               5|            0.1|      2|
|               a|           3|               1|            0.1|      3|
|               a|           4|               4|            0.1|      4|
|               a|           5|               5|            0.1|      5|
|               a|           6|               6|            0.1|      6|
|               a|           7|               8|            0.1|      1|
|               a|           8|               3|            0.1|      1|
|               a|           9|               1|            0.1|      2|
|               a|          10|               2|            0.1|      3|
|               b|           1|               4|   

In [65]:
C_R_BASE = 0.1
def create_APR(df, C_R_BASE):
    df = df.withColumn("1-c_r", (1 - F.col("commission_rate"))).orderBy("operator_address", "block_height")
    df = df.withColumn("APR", (1/(1-C_R_BASE) * (1- F.col("commission_rate"))))
    return df

def create_delta(df: DataFrame, block_num):
    w = Window.partitionBy('operator_address').orderBy('block_height').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    df = df.withColumn('subtract_de_token', F.when(
        (F.col("delegators_token")-F.first("delegators_token").over(w))>0 \
        ,F.col("delegators_token")-F.first("delegators_token").over(w))\
        .otherwise(0))
    
    df_delta = (
            df.groupBy("operator_address") \
                .agg({"subtract_de_token": "mean", \
                     "APR": "min"})
                .withColumnRenamed("avg(subtract_de_token)", "real_delta")\
                .withColumnRenamed("min(APR)", "APR")
    )
    df_delta = df_delta.withColumn("block_num", F.lit(block_num))
    

    return df_delta

def create_real_APR(real_df):
    df = real_df.withColumn("delta_APR_multiply", F.col("APR") * F.col("real_delta"))
    realAPR_df = df.groupBy("block_num") \
                .agg({"delta_APR_multiply": "sum", \
                       "real_delta": "sum"}) \
                .withColumnRenamed("sum(delta_APR_multiply)", "sum_delta_APR_multiply")\
                .withColumnRenamed("sum(real_delta)", "sum_real_delta")
    realAPR_df = realAPR_df.withColumn("real_APR", F.col("sum_delta_APR_multiply") / F.col("sum_real_delta"))
    return realAPR_df

def creat_unreal_APR(predict_df, realAPRdf, block_num):
    
       df_percen = predict_df.withColumn("block_num", F.lit(block_num))
       df_percen = df_percen.join(realAPRdf, on="block_num", how="left")
       df_percen = df_percen.withColumn("unreal_delta", F.col("predict") * F.col("sum_real_delta"))
       
       df_percen = df_percen.withColumn("unreal_delta_APR_multiply", F.col("APR") * F.col("unreal_delta"))

       unrealAPR_df = df_percen.groupBy("block_num") \
                     .agg({"unreal_delta_APR_multiply": "sum", \
                            "unreal_delta": "sum"}) \
                     .withColumnRenamed("sum(unreal_delta_APR_multiply)", "sum_unreal_delta_APR_multiply")\
                     .withColumnRenamed("sum(unreal_delta)", "sum_unreal_delta")
       unrealAPR_df = unrealAPR_df.withColumn("unreal_APR", F.col("sum_unreal_delta_APR_multiply") / F.col("sum_unreal_delta"))

       return unrealAPR_df

def join_df(real_df, predict_df):
    final_df = real_df.join(predict_df, on="block_num", how="right")
    final_df = final_df.drop("sum_delta_APR_multiply")
    final_df = final_df.drop("sum_real_delta")
    final_df = final_df.drop("sum_unreal_delta_APR_multiply")
    final_df = final_df.drop("sum_unreal_delta")
    return final_df

def back_test(df, C_R_BASE,start_block, end_block, timestamp_block):
    for i in range(start_block, end_block+1):
        if i == start_block:
            APR_df = create_APR(df, C_R_BASE)
            real_df = APR_df.filter(F.col("block_height").between(i, i+ timestamp_block))

            real_APR_df = create_delta(real_df, i)
            real_APR_df = create_real_APR(real_APR_df)

            predict_df = APR_df.filter(F.col("block_height") == i)
            unreal_APR_df = creat_unreal_APR(predict_df, real_APR_df, i)
            final_df = join_df(real_APR_df, unreal_APR_df)
        else:
            APR_df = create_APR(df, C_R_BASE)
            real_df = APR_df.filter(F.col("block_height").between(i, i+ timestamp_block))

            real_APR_df = create_delta(real_df, i)
            real_APR_df = create_real_APR(real_APR_df)

            predict_df = APR_df.filter(F.col("block_height") == i)
            unreal_APR_df = creat_unreal_APR(predict_df, real_APR_df, i)
            merge_df = join_df(real_APR_df, unreal_APR_df)
            final_df = final_df.union(merge_df)
    
    return final_df




In [61]:
df = create_APR(df, C_R_BASE)
real_df = df.filter(F.col("block_height").between(6, 6+ 1))

real_APR_df = create_delta(real_df, 6)
real_APR_df = create_real_APR(real_APR_df)

predict_df = df.filter(F.col("block_height") == 6)
unreal_APR_df = creat_unreal_APR(predict_df, real_APR_df, 6)
final_df6 = join_df(real_APR_df, unreal_APR_df)

In [52]:
final_df.show(200)

+---------+--------+------------------+
|block_num|real_APR|        unreal_APR|
+---------+--------+------------------+
|        1|     1.0|0.7777777777777778|
+---------+--------+------------------+



In [62]:
final_df6.show(200)

+---------+--------+------------------+
|block_num|real_APR|        unreal_APR|
+---------+--------+------------------+
|        6|     1.0|0.8461538461538461|
+---------+--------+------------------+



In [66]:
final_df = back_test(df, C_R_BASE, 1, 7,1)

In [67]:
final_df.show(200)

+---------+-----------------+------------------+
|block_num|         real_APR|        unreal_APR|
+---------+-----------------+------------------+
|        1|              1.0|0.7777777777777778|
|        2|0.888888888888889|0.8333333333333334|
|        3|              1.0|             0.875|
|        4|0.761904761904762|0.8492063492063491|
|        5|              1.0|0.8080808080808081|
|        6|              1.0|0.8461538461538461|
|        7|0.888888888888889|0.7901234567901235|
+---------+-----------------+------------------+



In [257]:
# def create_delta(df: DataFrame, label_win_size: int):
#     assert label_win_size % 150 == 0, "150 must be divisible by label_win_size"
#     size = int(label_win_size // 150) -1 # As we'll shift data after combining the data
    
#     ### window is used for shifting "size" blocks and calculate the mean
#     window = Window.partitionBy("operator_address").orderBy("block_height").rangeBetween(0, size)
#     w1 = Window.partitionBy("operator_address").orderBy("block_height").rangeBetween(0, 0)
    
#     ### lag_window is used for getting the null blocks - the last blocks that do not change
#     # lag_window = Window.partitionBy("operator_address").orderBy("block_height")
    
#     w=Window.partitionBy('operator_address').orderBy('block_height').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

#     df = df.withColumn(
#         "new_delta", F.sum(F.col("delegators_token")-F.first("delegators_token").over(w)).over(window)).orderBy("block_height")

#     # F.sum(F.when((F.col("delegators_token")-F.first("delegators_token").over(w))>0,F.col("delegators_token")-F.first("delegators_token").over(w)).otherwise(0)).over(window)).orderBy("block_height")

#     # df = df.withColumn("real_delta", (F.col("new_delta") - F.col("delegators_token")))
#     df = df.na.drop()
#     return df

In [258]:
# |-- block_height: integer (nullable = true)
#  |-- operator_address: string (nullable = true)
#  |-- jailed: integer (nullable = true)
#  |-- status: string (nullable = true)
#  |-- tokens: decimal(38,0) (nullable = true)
#  |-- commission_rate: double (nullable = true)
#  |-- self_bonded: decimal(38,6) (nullable = true)
#  |-- delegators_token: decimal(38,6) (nullable = true)
#  |-- validators_count_per_block: long (nullable = true)
#  |-- total_token_amt_per_block: decimal(38,0) (nullable = true)
#  |-- total_self_bonded_amt_per_block: decimal(38,6) (nullable = true)
#  |-- voting_power_score: decimal(38,6) (nullable = true)
#  |-- commission_score: double (nullable = true)
#  |-- self_bonded_score: double (nullable = true)
#  |-- vote_score: double (nullable = true)
#  |-- score: double (nullable = true)
#  |-- label: double (nullable = true)

In [16]:
list = ["oraivaloper18tf4uwrkcd4qk87jz3n0ruhsdzeg3fmdsdaw04",
"oraivaloper102rpf06zhyrfps4qzvehqjtr7rh52us5z2959t",
"oraivaloper10z9f6539v0ge78xlm4yh7tddrvw445s6d7s2xq",
"oraivaloper1zwxeq68hunufljymhy0f0x7r2vvh9mu2hh8nt8",
"oraivaloper1yujklf8xlcngsdylgqj352pj9x3yvrrwf3spc8",
"oraivaloper1mrv57zj3dpfyc9yd5xptnz2tqfez9fss4c9r85",
"oraivaloper17zr98cwzfqdwh69r8v5nrktsalmgs5sa83gpd9",
"oraivaloper1n4xkr4k4rpynp020e8xfqwm2e95g7rzx7w6kgl",
"oraivaloper1nedr7579x9dp6s3rrt7n5a5s5qczpnnstanuk5",
"oraivaloper16spaytw7nhdrv9lar0quwj6eex8cw9pvmtynxd",
"oraivaloper1asz5wl5c2xt8y5kyp9r04v54zh77pq90qar7e8",
"oraivaloper1djm07np8dzyg4et3d7dqtr3692l80nggvl0edh",
"oraivaloper1ucx0gm8kca2zvyr9d39z249j62y2t8r0rwtmr6",
"oraivaloper1ch3ewye24zm094ygmxu5e4z7d0xre3vhthctpn",
"oraivaloper1w96q7rrd6edc3fpmw6zayzdf5ajzucrnw3pt3s",
"oraivaloper1v26tdegnk79edw7xkk2xh8qn89vy6qej6yhsev",
"oraivaloper1kh9vejqxqqccavtv2nf683mx0z85mfpd7q566q",
"oraivaloper1d2ths86a84rxpp7s9j64qgrypntw2pn7lndfgs",
"oraivaloper1xsptthm2ylfw0salut97ldfan2jt032nye7s00",
"oraivaloper1qv5jn7tueeqw7xqdn5rem7s09n7zletreera88",
"oraivaloper16e6cpk6ycddk6208fpaya7tmmardhvr7h40yqy",
"oraivaloper10n3fs6fkl4fp9dcsdfl2vl3ay7pk7snntfg7da",
"oraivaloper130jsl66rgss6eq7qur02yfr6tzppdvxglz7n7g",
"oraivaloper1yc9nysml8dxy447hp3aytr0nssr9pd9a47l7gx",
"oraivaloper109vcny07r3waj9sld4ejasjyal0rudskeax7uc",
"oraivaloper1h89umsrsstyeuet8kllwvf2tp630n77aymck78",
"oraivaloper1mxqeldsxg60t2y6gngpdm5jf3k96dnju5el96f",
"oraivaloper1rqq57xt5r5pnuguffcrltnvkul7n0jdxxdgey0",
"oraivaloper1uhcwtfntsvk8gpwfxltesyl4e28aalmq9v9z0x",
"oraivaloper1xesqr8vjvy34jhu027zd70ypl0nnev5euy9nyl",
"oraivaloper1cp0jml5fxkdvmajcwvkue9d0sym6s0vqly88hg",
"oraivaloper1u2344d8jwtsx5as7u5jw7vel28puh34q7d3y64",
"oraivaloper1f6q9wjn8qp3ll8y8ztd8290vtec2yxyxxygyy2",
"oraivaloper1h9gg3xavqdau6uy3r36vn4juvzsg0lqvszgtvc",
"oraivaloper14nz2pqskfv9kcez8u0a9gnnsgwjerzqxpmne0y",
"oraivaloper1m2d5uhr65p9vvlw2w29kajud5q529a76v22wyu",
"oraivaloper13ckyvg0ah9vuujtd49yner2ky92lej6nwjvrjv",
"oraivaloper12ru3276mkzuuay6vhmg3t6z9hpvrsnplm2994n",
"oraivaloper14vcw5qk0tdvknpa38wz46js5g7vrvut8ku5kaa"]

In [74]:
df = spark.read.parquet("data/percent.parquet")

In [78]:
df.orderBy(F.asc("block_height")).show(200)


+------------+--------------------+----------------+--------------------+-------------------+--------------------+------------------+--------------------+-------------------+----------+------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+--------------------------+-------------------+----------------+----------------+-------------------------+------------------------+------------------------+--------------------------+-------------------------+-------------------------+-------------------+------------------+------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+--------------------------+-------------------+----------------+---------------+-------------------------+------------------------+------------------------+--------------------------+-------------------------+-------------------------+-----

In [75]:
df.orderBy

23/01/30 11:32:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+--------------------+----------------+--------------------+-------------------+--------------------+------------------+--------------------+-------------------+----------+------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+--------------------------+-------------------+----------------+----------------+-------------------------+------------------------+------------------------+--------------------------+-------------------------+-------------------------+-------------------+------------------+------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+--------------------------+-------------------+----------------+---------------+-------------------------+------------------------+------------------------+--------------------------+-------------------------+-------------------------+-----

In [80]:
for i in range(1, 8, 2):
    print(i)

1
3
5
7
